In [ ]:
!pip install pandas==2.2.2 numpy pyspark==3.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 63.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285391 sha256=faa7921187803296fc466cf484cf5c537e8a5fca637745f4e8c72f1af351a6f8
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 2.2.2 which is 

In [ ]:
# Contoh membuat DataFrame sederhana dan operasi dasar
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HandsOnPertemuan6').getOrCreate()

data = [('Nasywaa', 'pilot', 3000),
        ('Eka', 'dokter', 4600),
        ('Dhiya', 'guru', 4100),
        ('Risma', 'Bidan', 3000)]
columns = ['EmployeeName', 'Department', 'Salary']

df = spark.createDataFrame(data, schema=columns)
df.show()

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|     Nasywaa|     pilot|  3000|
|         Eka|    dokter|  4600|
|       Dhiya|      guru|  4100|
|       Risma|     Bidan|  3000|
+------------+----------+------+



In [ ]:
# Contoh operasi transformasi DataFrame
df.select('EmployeeName', 'Salary').show()
df.filter(df['Salary'] > 3000).show()
df.groupBy('Department').avg('Salary').show()

+------------+------+
|EmployeeName|Salary|
+------------+------+
|     Nasywaa|  3000|
|         Eka|  4600|
|       Dhiya|  4100|
|       Risma|  3000|
+------------+------+

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|         Eka|    dokter|  4600|
|       Dhiya|      guru|  4100|
+------------+----------+------+

+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|     pilot|     3000.0|
|    dokter|     4600.0|
|     Bidan|     3000.0|
|      guru|     4100.0|
+----------+-----------+



In [ ]:
# 3 Menambahkan kolom bonus
df_with_bonus = df.withColumn('SalaryBonus', df['Salary'] * 0.1)

# Menambahkan kolom TotalCompensation menggunakan df_with_bonus
df_with_total = df_with_bonus.withColumn('TotalCompensation', df_with_bonus['Salary'] + df_with_bonus['SalaryBonus'])

# Menampilkan DataFrame dengan kolom baru
df_with_total.show()

+------------+----------+------+-----------+-----------------+
|EmployeeName|Department|Salary|SalaryBonus|TotalCompensation|
+------------+----------+------+-----------+-----------------+
|     Nasywaa|     pilot|  3000|      300.0|           3300.0|
|         Eka|    dokter|  4600|      460.0|           5060.0|
|       Dhiya|      guru|  4100|      410.0|           4510.0|
|       Risma|     Bidan|  3000|      300.0|           3300.0|
+------------+----------+------+-----------+-----------------+



In [ ]:
# Contoh menggunakan window functions
from pyspark.sql.window import Window
from pyspark.sql import functions as F

windowSpec = Window.partitionBy('Department').orderBy('Salary')
df.withColumn('Rank', F.rank().over(windowSpec)).show()

+------------+----------+------+----+
|EmployeeName|Department|Salary|Rank|
+------------+----------+------+----+
|       Risma|     Bidan|  3000|   1|
|         Eka|    dokter|  4600|   1|
|       Dhiya|      guru|  4100|   1|
|     Nasywaa|     pilot|  3000|   1|
+------------+----------+------+----+

